In [1]:
!pip install konlpy

In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import re

In [3]:
data = pd.read_csv('/content/drive/My Drive/ai_school/navermovies2.csv')

data.shape

(17320, 12)

In [4]:
data.head()

,Unnamed: 0,순위,제목,포스터,평점,"나라,상영시간",개봉일,등급,장르,감독,주연,줄거리
0,0,0,너의 이름은.,https://movie-phinf.pstatic.net/20171222_70/15...,8.78,"일본, 106분",2017.01.04.,12세 관람가,드라마,신카이 마코토,"지창욱, 김소현, 이레, 카미키 류노스케, 카미시라이시 모네, 타니 카논","꿈 속에서 몸이 뒤바뀐 도시 소년 ‘타키’와 시골 소녀 ‘미츠하’, 만난 적 없는 ..."
1,1,1,라라랜드,https://movie-phinf.pstatic.net/20161123_188/1...,8.60,"미국, 127분",2016.12.07.,12세 관람가,드라마,다미엔 차젤레,"엠마 스톤, 라이언 고슬링",꿈을 꾸는 사람들을 위한 별들의 도시 ‘라라랜드’. 재즈 피아니스트 ‘세바스찬’(라...
2,2,2,증인,https://movie-phinf.pstatic.net/20190226_194/1...,9.09,"대한민국, 129분",2019.02.13.,12세 관람가,드라마,이한,"정우성, 김향기",“목격자가 있어. 자폐아야” 신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음...
3,3,3,청년경찰,https://movie-phinf.pstatic.net/20170814_131/1...,8.26,"대한민국, 109분",2017.08.09.,15세 관람가,액션,김주환,"박서준, 강하늘",의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘) 둘도 없는 친구...
4,4,4,돈,https://movie-phinf.pstatic.net/20190306_280/1...,7.41,"대한민국, 115분",2019.03.20.,15세 관람가,액션,박누리,"류준열, 유지태, 조우진",오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(...


In [5]:
# 0번째 열은 순위와 같은 값을 갖는 중복된 열이므로 제외
data = data.iloc[:, 1:] 

In [6]:
# 나라와 상영시간을 분리하여 각각의 열 생성
data[['나라', '상영시간']] = data['나라,상영시간'].str.split(', ', expand=True) 
# 나라와 상영시간이 같이 있던 열은 삭제
del data['나라,상영시간'] 

data.head()

,순위,제목,포스터,평점,개봉일,등급,장르,감독,주연,줄거리,나라,상영시간
0,0,너의 이름은.,https://movie-phinf.pstatic.net/20171222_70/15...,8.78,2017.01.04.,12세 관람가,드라마,신카이 마코토,"지창욱, 김소현, 이레, 카미키 류노스케, 카미시라이시 모네, 타니 카논","꿈 속에서 몸이 뒤바뀐 도시 소년 ‘타키’와 시골 소녀 ‘미츠하’, 만난 적 없는 ...",일본,106분
1,1,라라랜드,https://movie-phinf.pstatic.net/20161123_188/1...,8.60,2016.12.07.,12세 관람가,드라마,다미엔 차젤레,"엠마 스톤, 라이언 고슬링",꿈을 꾸는 사람들을 위한 별들의 도시 ‘라라랜드’. 재즈 피아니스트 ‘세바스찬’(라...,미국,127분
2,2,증인,https://movie-phinf.pstatic.net/20190226_194/1...,9.09,2019.02.13.,12세 관람가,드라마,이한,"정우성, 김향기",“목격자가 있어. 자폐아야” 신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음...,대한민국,129분
3,3,청년경찰,https://movie-phinf.pstatic.net/20170814_131/1...,8.26,2017.08.09.,15세 관람가,액션,김주환,"박서준, 강하늘",의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘) 둘도 없는 친구...,대한민국,109분
4,4,돈,https://movie-phinf.pstatic.net/20190306_280/1...,7.41,2019.03.20.,15세 관람가,액션,박누리,"류준열, 유지태, 조우진",오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(...,대한민국,115분


In [7]:
data['장르'].unique() # 장르에 무언가 오류가 있음을 확인

array(['드라마', '액션', '코미디', 'SF/판타지', '다큐멘터리', '이석근', '프랭키 첸', '공포/스릴러',
       '애니메이션', '테아 샤록', '이용주', '스콧 스피어', '코이즈미 노리히로', '서은영', '허진호', '백',
       '이장훈', '김한결', '설효로', '김종관', '사준의', '이윤기', '임대형', '송일곤', '신조 타케히코',
       '기타가와 에리코', '케네스 브래너', '공연실황', '레베카 밀러', '정재은', '빅터 레빈', '한철수',
       '츠키카와 쇼', '휴가 아사코', '김대승', '츠츠미 유키히코', '주동, 대몽영', '쿠마자와 나오토',
       '타카타 마사히로', '카와무라 타이스케', '타구치 토모로오', '크리스티안 디터', '우디 앨런', '이창근',
       '미키 코이치로', '시바야마 겐지', '마이클 수지', '최호', '마이클 쇼월터', '안정민', '앨런 릭먼',
       '야쿠모 사이지', '데이빗 맥킨지', '유위강', '이윤정', '이누도 잇신', '김기훈', '독립영화',
       '이시이 야스하루', '김정권', '백승환', '조성은', '소륜', '마크 레빈', '트란 안 훙', '사토 유이치',
       '레스티 첸', '후루사와 켄', '히로키 류이치', '미조구치 미노루', '피에르 살바도리', '이구치 노보루',
       '김용완, 선종훈', '잭 스마이트', '김희진', '사노 토모키', '야니스 스마라그디스', '황다슬',
       '데이빗 린', '한상희', '유영의', '장 콕토', '리 위', '이강현', '윤가은', '최현영',
       '루치노 비스콘티', '조성규', '세드릭 클라피쉬', '민병우', '이와이 슌지', '나카니시 켄지', '하몽화',
       '이송희일', '람지 베디아', '박민국', '후고 데 아나', '로버트 돈헴', '장률', '멜로', '이미지',
   

In [8]:
genre_count = data['장르'].value_counts()
genre_count.index[:15] # 0번째('액션')부터 9번째('멜로')까지가 실제 장르인 것으로 생각된다.

Index(['액션', '드라마', '코미디', '공포/스릴러', '애니메이션', '공연실황', '다큐멘터리', 'SF/판타지',
       '독립영화', '멜로', '유영의', '한상희', '이윤정', '김한결', '헨리 헤서웨이'],
      dtype='object')

In [9]:
genres = genre_count.index[:10].tolist() # 장르 목록 리스트화
genres

['액션',
 '드라마',
 '코미디',
 '공포/스릴러',
 '애니메이션',
 '공연실황',
 '다큐멘터리',
 'SF/판타지',
 '독립영화',
 '멜로']

In [10]:
# 장르가 genres에 포함되지 않는 행 삭제
data = data[data['장르'].isin(genres)]
data.shape

(17017, 12)

In [11]:
data['장르'].unique()

array(['드라마', '액션', '코미디', 'SF/판타지', '다큐멘터리', '공포/스릴러', '애니메이션', '공연실황',
       '독립영화', '멜로'], dtype=object)

In [12]:
data.isnull().sum()

순위         0
제목         0
포스터        0
평점         0
개봉일     6963
등급         0
장르         0
감독         0
주연        61
줄거리      747
나라         0
상영시간       0
dtype: int64

In [13]:
# 줄거리가 '줄거리 준비중입니다. / 줄거리 정보가 없습니다. / 줄거리 정보가 없습니다'인 행 삭제
no_summary = ['줄거리 준비중입니다.', '줄거리 정보가 없습니다.', '줄거리 정보가 없습니다']
data = data[~data['줄거리'].isin(no_summary)]
data.shape

(16077, 12)

In [14]:
# '평점'이 4 이하인 행 삭제
data = data[data['평점'] > 4]
data.shape

(13530, 12)

In [15]:
# '주연'과 '줄거리'가 null 값인 행 삭제
data = data.dropna(subset=['주연', '줄거리'])
data.shape

(13097, 12)

In [16]:
data.isnull().sum()

순위         0
제목         0
포스터        0
평점         0
개봉일     3629
등급         0
장르         0
감독         0
주연         0
줄거리        0
나라         0
상영시간       0
dtype: int64

In [17]:
# '감독'과 '주연'의 값을 공백을 기준으로 나누는 하나의 문자열화
# 이름이 두 어절 이상인 사람들을 하나로 인지가히기 위해 먼저 공백을 없앤 후, 쉼표를 공백으로 replace 한다.
data['감독'] = data['감독'].str.replace(' ', '').str.replace(',', ' ')
data['주연'] = data['주연'].str.replace(' ', '').str.replace(',', ' ')
data.head()

,순위,제목,포스터,평점,개봉일,등급,장르,감독,주연,줄거리,나라,상영시간
0,0,너의 이름은.,https://movie-phinf.pstatic.net/20171222_70/15...,8.78,2017.01.04.,12세 관람가,드라마,신카이마코토,지창욱 김소현 이레 카미키류노스케 카미시라이시모네 타니카논,"꿈 속에서 몸이 뒤바뀐 도시 소년 ‘타키’와 시골 소녀 ‘미츠하’, 만난 적 없는 ...",일본,106분
1,1,라라랜드,https://movie-phinf.pstatic.net/20161123_188/1...,8.60,2016.12.07.,12세 관람가,드라마,다미엔차젤레,엠마스톤 라이언고슬링,꿈을 꾸는 사람들을 위한 별들의 도시 ‘라라랜드’. 재즈 피아니스트 ‘세바스찬’(라...,미국,127분
2,2,증인,https://movie-phinf.pstatic.net/20190226_194/1...,9.09,2019.02.13.,12세 관람가,드라마,이한,정우성 김향기,“목격자가 있어. 자폐아야” 신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음...,대한민국,129분
3,3,청년경찰,https://movie-phinf.pstatic.net/20170814_131/1...,8.26,2017.08.09.,15세 관람가,액션,김주환,박서준 강하늘,의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘) 둘도 없는 친구...,대한민국,109분
4,4,돈,https://movie-phinf.pstatic.net/20190306_280/1...,7.41,2019.03.20.,15세 관람가,액션,박누리,류준열 유지태 조우진,오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(...,대한민국,115분


In [18]:
def split_data(x):
    nlp = Okt()
    try:
        nouns = nlp.nouns(x)
        nouns = [n for n in nouns if len(n) > 1]
        return nouns

    except:
        return 'None'

In [19]:
# '줄거리'에 위에서 정의한 split_data 함수를 적용해 'keywords'를 만든 후, 위에서와 같이 하나의 문자열화
data['keywords'] = data['줄거리'].apply(split_data)
data['keywords'] = data['keywords'].apply(lambda x : ' '.join(x))

data.head()

,순위,제목,포스터,평점,개봉일,등급,장르,감독,주연,줄거리,나라,상영시간,keywords
0,0,너의 이름은.,https://movie-phinf.pstatic.net/20171222_70/15...,8.78,2017.01.04.,12세 관람가,드라마,신카이마코토,지창욱 김소현 이레 카미키류노스케 카미시라이시모네 타니카논,"꿈 속에서 몸이 뒤바뀐 도시 소년 ‘타키’와 시골 소녀 ‘미츠하’, 만난 적 없는 ...",일본,106분,도시 소년 타키 시골 소녀 미츠하 만난 사람 기적 사랑 이야기 이름 사람 사람 이름
1,1,라라랜드,https://movie-phinf.pstatic.net/20161123_188/1...,8.60,2016.12.07.,12세 관람가,드라마,다미엔차젤레,엠마스톤 라이언고슬링,꿈을 꾸는 사람들을 위한 별들의 도시 ‘라라랜드’. 재즈 피아니스트 ‘세바스찬’(라...,미국,127분,사람 도시 라라 랜드 재즈 피아니스트 세바스찬 라이언 슬링 배우 지망 미아 엠마 스...
2,2,증인,https://movie-phinf.pstatic.net/20190226_194/1...,9.09,2019.02.13.,12세 관람가,드라마,이한,정우성 김향기,“목격자가 있어. 자폐아야” 신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음...,대한민국,129분,목격자 자폐 아야 신념 잠시 현실 위해 속물 민변 출신 대형 로펌 변호사 정우성 파...
3,3,청년경찰,https://movie-phinf.pstatic.net/20170814_131/1...,8.26,2017.08.09.,15세 관람가,액션,김주환,박서준 강하늘,의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘) 둘도 없는 친구...,대한민국,109분,의욕 충만 경찰대 기준 박서준 이론 경찰대 강하늘 친구 사람 외출 납치 사건 목격 ...
4,4,돈,https://movie-phinf.pstatic.net/20190306_280/1...,7.41,2019.03.20.,15세 관람가,액션,박누리,류준열 유지태 조우진,오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(...,대한민국,115분,오직 부자 여의도 증권 입성 신입 주식 브로커 조일현 류준열 수수료 해고 직전 처지...


In [20]:
# 순위(인덱스) 순으로 정렬 후, 0부터 시작하는 순위에 1씩 더한다.
data = data.sort_index()
data['순위'] += 1

In [21]:
# 제목이 같은 중복 데이터 삭제
data = data.drop_duplicates(subset='제목')

In [22]:
# 인덱스 재설정
data = data.reset_index(drop=True)

# 모델

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval

## CounterVectorizer - 장르, 감독, 주연, keywords를 고려

In [24]:
# '장르', '감독', '주연', 'keywords'를 하나로 벡터화
mv = data['장르'] + ' ' +  data['감독'] + ' ' + data['주연'] + ' ' + data['keywords']

vec = CountVectorizer(ngram_range=(1, 2))
matrix = vec.fit_transform(mv)

In [25]:
similarity = cosine_similarity(matrix, matrix)
print(similarity)

count_similar_index = np.argsort(-similarity)
print(count_similar_index)

[[1.         0.18521695 0.02437756 ... 0.02321817 0.01703748 0.02864166]
 [0.18521695 1.         0.01196511 ... 0.02279212 0.01672484 0.02811608]
 [0.02437756 0.01196511 1.         ... 0.01499906 0.0110063  0.01850266]
 ...
 [0.02321817 0.02279212 0.01499906 ... 1.         0.0209657  0.03524537]
 [0.01703748 0.01672484 0.0110063  ... 0.0209657  1.         0.02586303]
 [0.02864166 0.02811608 0.01850266 ... 0.03524537 0.02586303 1.        ]]
[[   0 5757 5464 ... 4045 6328 2605]
 [   1    0 8209 ... 6141 6096 5689]
 [   2  663 8242 ... 2025 6024 6294]
 ...
 [8974 1200 6037 ...  649 4354  557]
 [8975 8352 7934 ... 6701 6691 7201]
 [8976 1696 6308 ... 4368 4393  147]]


## TfidfVectorizer 사용 - 주연을 기준으로

In [26]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(data['주연'])

In [27]:
actor_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(actor_similarity)

tfidf_similar_index = np.argsort(-actor_similarity)
print(tfidf_similar_index)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[   0  414 5200 ... 3001 2995 8976]
 [   1 1782  859 ... 3004 2997 8976]
 [ 663    2  870 ... 3011 3005 8976]
 ...
 [8974 6747    0 ... 2994 2988 8976]
 [8975 7500 8218 ... 2994 2988 8976]
 [8976 8213 8787 ... 2994 2988 4488]]


# 예측기

In [28]:
# '장르, 감독, 주연, keywords'를 벡터화한 CountVectorizer
input_movie = input()

movie_index = data[data['제목'] == input_movie].index.values
print(movie_index)
similar_movies = count_similar_index[movie_index, 0:10]
print(similar_movies)

similar_movies_index = similar_movies.reshape(-1)
print(similar_movies_index)
print(data.iloc[similar_movies_index, 1:4])

어바웃 타임
[9]
[[   9 4948   27 1388 3875 4628 1952 2051 5035 1741]]
[   9 4948   27 1388 3875 4628 1952 2051 5035 1741]
                 제목                                                포스터    평점
9            어바웃 타임  https://movie-phinf.pstatic.net/20131115_243/1...  9.18
4948  단지 키스, 맛있는 키스  https://movie-phinf.pstatic.net/20150225_132/1...  9.17
27          러브 액츄얼리  https://movie-phinf.pstatic.net/20191202_278/1...  8.94
1388   당신이 사랑하는 동안에  https://movie-phinf.pstatic.net/20161020_217/1...  8.47
3875      왕대륙의 청춘로드  https://movie-phinf.pstatic.net/20171227_152/1...  7.93
4628       인생은 새옹지마  https://movie-phinf.pstatic.net/20140529_151/1...  7.41
1952        고스트 스토리  https://movie-phinf.pstatic.net/20171207_182/1...  7.54
2051           음식남녀  https://movie-phinf.pstatic.net/20170518_96/14...  8.25
5035            인형사  https://movie-phinf.pstatic.net/20111222_126/1...  6.32
1741          스타더스트  https://movie-phinf.pstatic.net/20111222_283/1...  8.80


In [29]:
# '주연'으로 만든 TfidfVectorizer
input_movie = input()

movie_index = data[data['제목'] == input_movie].index.values
print(movie_index)
similar_movies = tfidf_similar_index[movie_index, 0:10]
print(similar_movies)

similar_movies_index = similar_movies.reshape(-1)
print(similar_movies_index)
print(data.iloc[similar_movies_index, 1:4])

어바웃 타임
[9]
[[   9  679 3311  917 2775 1731 1550   16  565  626]]
[   9  679 3311  917 2775 1731 1550   16  565  626]
                    제목                                                포스터    평점
9               어바웃 타임  https://movie-phinf.pstatic.net/20131115_243/1...  9.18
679                프랭크  https://movie-phinf.pstatic.net/20140915_194/1...  7.94
3311       에브리띵 윌 비 파인  https://movie-phinf.pstatic.net/20151209_264/1...  6.15
917                 서약  https://movie-phinf.pstatic.net/20120214_174/1...  8.22
2775          나이트 플라이트  https://movie-phinf.pstatic.net/20111222_179/1...  7.21
1731            디서비디언스  https://movie-phinf.pstatic.net/20180307_56/15...  7.55
1550        시간 여행자의 아내  https://movie-phinf.pstatic.net/20170309_252/1...  8.28
16                 노트북  https://movie-phinf.pstatic.net/20161021_141/1...  9.04
565   레버넌트: 죽음에서 돌아온 자  https://movie-phinf.pstatic.net/20151229_126/1...  7.84
626           굿모닝 에브리원  https://movie-phinf.pstatic.net/20111223_88/13...  8.28
